## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-26-20-50-00 +0000,wsj,Opinion | Notable & Quotable: Crime in Chicago,https://www.wsj.com/opinion/notable-quotable-c...
1,2025-08-26-20-49-01 +0000,nyt,SpaceX Scrubs 10th Starship Test Launch Becaus...,https://www.nytimes.com/2025/08/25/science/spa...
2,2025-08-26-20-47-18 +0000,startribune,"Live: Who’s in, who’s out for the Vikings on r...",https://www.startribune.com/minnesota-vikings-...
3,2025-08-26-20-46-18 +0000,nyt,SpaceX Starship Test Launch: How and When to W...,https://www.nytimes.com/2025/08/26/science/spa...
4,2025-08-26-20-44-13 +0000,startribune,"1 dead, 6 injured in south Minneapolis shootin...",https://www.startribune.com/police-investigati...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2413/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
60,trump,69
63,cook,20
62,lisa,18
75,gaza,16
325,new,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
220,2025-08-26-03-44-00 +0000,wsj,"President Trump said he is removing Lisa Cook,...",https://www.wsj.com/politics/policy/trump-says...,184
189,2025-08-26-09-30-00 +0000,wsj,President Trump’s attempt to fire Federal Rese...,https://www.wsj.com/economy/central-banking/un...,163
22,2025-08-26-20-04-00 +0000,wsj,President Trump has told advisers that he want...,https://www.wsj.com/politics/policy/trump-weig...,154
114,2025-08-26-16-12-46 +0000,wapo,Federal Reserve governor Lisa Cook says she’ll...,https://www.washingtonpost.com/business/2025/0...,153
110,2025-08-26-16-22-37 +0000,nyt,Can Trump Fire a Fed Governor? What to Know as...,https://www.nytimes.com/2025/08/22/us/politics...,153


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
220,184,2025-08-26-03-44-00 +0000,wsj,"President Trump said he is removing Lisa Cook,...",https://www.wsj.com/politics/policy/trump-says...
149,61,2025-08-26-13-35-35 +0000,nypost,Netanyahu calls deadly double strike on Gaza h...,https://nypost.com/2025/08/26/world-news/netan...
160,55,2025-08-26-11-52-00 +0000,wsj,Illinois Democratic Gov. JB Pritzker promised ...,https://www.wsj.com/politics/policy/chicago-vo...
168,49,2025-08-26-10-39-53 +0000,cbc,Australia accuses Iran of being behind two ant...,https://www.cbc.ca/news/world/australia-iran-a...
29,46,2025-08-26-19-53-01 +0000,nypost,Judge rips Andrew Cuomo for using $20M in taxp...,https://nypost.com/2025/08/26/us-news/judge-ri...
150,36,2025-08-26-13-33-45 +0000,nypost,Syrian migrant arrested for brutal attack on A...,https://nypost.com/2025/08/26/world-news/syria...
158,34,2025-08-26-12-16-00 +0000,wsj,The U.S. will increase tariffs and impose expo...,https://www.wsj.com/politics/policy/trump-thre...
3,34,2025-08-26-20-46-18 +0000,nyt,SpaceX Starship Test Launch: How and When to W...,https://www.nytimes.com/2025/08/26/science/spa...
26,33,2025-08-26-19-59-48 +0000,nyt,Trump’s Attempt to Fire the Fed’s Lisa Cook Ri...,https://www.nytimes.com/2025/08/26/us/politics...
88,33,2025-08-26-17-25-43 +0000,nypost,NY’s secret ‘backroom deal’ over one LI school...,https://nypost.com/2025/08/26/us-news/nys-ditc...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
